# <font color=#f4665a>  ICA analysis </font>

### Project: BMPD HC
____________________________________________________

**Description:** This notebook provides code for BOLD signal fMRI resting-state processing for the Biomarker for Parkinson's Disease (BMPD)data. 
We will used ICA analysis:
CanICA is an ICA package for group-level analysis of fMRI data.  
It brings a well-controlled group model, as well as a thresholding algorithm controlling for specificity and sensitivity with an explicit model of the signal.  
The reference papers are: G. Varoquaux et al. "A group model for stable multi-subject ICA on fMRI datasets", NeuroImage Vol 51 (2010), p. 288-299
G. Varoquaux et al. "ICA-based sparse features recovery from fMRI datasets", IEEE ISBI 2010, p. 1177


**Toolbox required:** SpinalCordToolbox, FSL, nilearn toolbox, nipype, matlab

**Inputs**:  
This notebook required this the following prepross anatomical,fmri images 

**Ouputs**:
See the output description at each step of the Notebook.

____________________________________________________


## <font color=#00988c> Imports </font>

In [55]:
import sys
import json
# Spinal cord Toolbox_________________________________________
### Cerebro:
sys.path.append("/cerebro/cerebro1/dataset/bmpd/derivatives/thibault_test/code/toolbox/spinalcordtoolbox-5.0.0")
sys.path.append("/cerebro/cerebro1/dataset/bmpd/derivatives/thibault_test/code/toolbox/spinalcordtoolbox-5.0.0/scripts") #sys.path.insert(0, "/cerebro/cerebro1/dataset/bmpd/derivatives/sc_preproc/code/sct/spinalcordtoolbox")
sys.path.append("/cerebro/cerebro1/dataset/bmpd/derivatives/HealthyControls_project/hc_project_analyses/code/") #sys.path.insert(0, "/cerebro/cerebro1/dataset/bmpd/derivatives/sc_preproc/code/sct/spinalcordtoolbox")

from spinalcordtoolbox.utils.sys import run_proc

from canICA_analyses import ICA
%matplotlib inline
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## <font color=#00988c>  Run the ICA analysis </font>

In [56]:
# Load the dataset config
with open('/cerebro/cerebro1/dataset/bmpd/derivatives/HealthyControls_project/hc_project_analyses/config/config_SPiCiCAP_CL.json') as config_file:
    config = json.load(config_file)
    
for k in config["k_range"]["spinalcord"]:
    config["n_comp"]=k # usefull if you want to test only on k
    print(config["n_comp"])
    icas = ICA("spinalcord",config) # "brain_spinalcord" or "brain" or "spinalcord"
    #all_data=icas.get_data(run='extract',t_r=config["t_r"],n_jobs=8) # load or extract
    all_data=icas.get_data(run='load',t_r=config["t_r"],n_jobs=8) # load or extract
    reducedata_all=icas.indiv_PCA(all_data,save_indiv_img=True)
    components=icas.get_CCA(reducedata_all)
    components_final,components_final_z=icas.get_ICA(components)
    zcomponents4D_filename=icas.save_components(components_final,components_final_z)
    
    

5
Analyse will be run on spinalcord structure
Mask: /cerebro/cerebro1/dataset/bmpd/derivatives/HealthyControls_project//templates/PAM50/PAM50_SPiCiCAP/template/PAM50_cord.nii.gz


OSError: [Errno 12] Cannot allocate memory

In [39]:
from nilearn.maskers import NiftiMasker
nifti_masker =  NiftiMasker(mask_img='/cerebro/cerebro1/dataset/bmpd/derivatives/HealthyControls_project//templates/PAM50/PAM50_SPiCiCAP/template/PAM50_cord_resampled.nii.gz')
new =nifti_masker.fit_transform('/cerebro/cerebro1/dataset/caroline/2022_SPiCiCAP_NK_CL/CL_analyses/denoising/2_func_in_template/sub-01/spinalcord/sub-01_fmri_moco_outliers_2reg_HP01_std_sc_inTemplate.nii.gz')
fmri_masked=nifti_masker.inverse_transform(new)

#fmri_masked = nifti_masker.transform('/cerebro/cerebro1/dataset/caroline/2022_SPiCiCAP_NK_CL/CL_analyses/denoising/2_func_in_template/sub-01/spinalcord/sub-01_fmri_moco_outliers_2reg_HP01_std_sc_inTemplate.nii.gz')
#fmri_masked = nifti_masker.inverse_transform('/cerebro/cerebro1/dataset/caroline/2022_SPiCiCAP_NK_CL/CL_analyses/denoising/2_func_in_template/sub-01/spinalcord/sub-01_fmri_moco_outliers_2reg_HP01_std_sc_inTemplate.nii.gz')
fmri_masked.to_filename('/cerebro/cerebro1/dataset/caroline/2022_SPiCiCAP_NK_CL/CL_analyses/denoising/2_func_in_template/sub-01/spinalcord/sub-01_fmri_moco_outliers_2reg_HP01_std_sc_inTemplate_masked.nii.gz') #save the n components for each subjects


In [122]:
components.shape # 73794,5

(73794, 5)

In [3]:
 zcomponents4D_filename=icas.save_components(components_final,components_final_z)

## <font color=#00988c>  Create masks </font>
### <font color=#00988c>  > Binarize IC maps </font> #_nindiv40

In [10]:
import glob
sys.path.append("/cerebro/cerebro1/dataset/bmpd/derivatives/thibault_test/code/toolbox/spinalcordtoolbox-5.0.0")
sys.path.append("/cerebro/cerebro1/dataset/bmpd/derivatives/thibault_test/code/toolbox/spinalcordtoolbox-5.0.0/scripts") #sys.path.insert(0, "/cerebro/cerebro1/dataset/bmpd/derivatives/sc_preproc/code/sct/spinalcordtoolbox")
from spinalcordtoolbox.utils.sys import run_proc
structure='spinalcord'
import os
for K in range(9,10):
    input_dir=config["main_dir"]+config["ica_root"]+ '/'+structure +'/K_'+ str(K) + '/comp_zscored/'
    output_dir=config["main_dir"]+config["ica_root"]+ '/'+structure +'/K_'+ str(K) + '/comp_bin/'
    for k in range(len(sorted(glob.glob(input_dir + '*_k_*')))):
        input_image=sorted(glob.glob(input_dir + '*_k_*'))[k]
        output_image=output_dir+os.path.basename(input_image).split('.')[0]+'_bin.nii.gz'
        run_proc('fslmaths {} -thr 2 -bin {}'.format(input_image,output_image)) # thre and binarize the image

fslmaths /cerebro/cerebro1/dataset/bmpd/derivatives/HealthyControls_project//ICA//spinalcord/K_9/comp_zscored/zCanICA_22sbj_spinalcord_spinalcord_k_1.nii.gz -thr 2 -bin /cerebro/cerebro1/dataset/bmpd/derivatives/HealthyControls_project//ICA//spinalcord/K_9/comp_bin/zCanICA_22sbj_spinalcord_spinalcord_k_1_bin.nii.gz # in /cerebro/cerebro1/dataset/bmpd/derivatives/HealthyControls_project/ICA/code
fslmaths /cerebro/cerebro1/dataset/bmpd/derivatives/HealthyControls_project//ICA//spinalcord/K_9/comp_zscored/zCanICA_22sbj_spinalcord_spinalcord_k_2.nii.gz -thr 2 -bin /cerebro/cerebro1/dataset/bmpd/derivatives/HealthyControls_project//ICA//spinalcord/K_9/comp_bin/zCanICA_22sbj_spinalcord_spinalcord_k_2_bin.nii.gz # in /cerebro/cerebro1/dataset/bmpd/derivatives/HealthyControls_project/ICA/code
fslmaths /cerebro/cerebro1/dataset/bmpd/derivatives/HealthyControls_project//ICA//spinalcord/K_9/comp_zscored/zCanICA_22sbj_spinalcord_spinalcord_k_3.nii.gz -thr 2 -bin /cerebro/cerebro1/dataset/bmpd/deri

### <font color=#00988c>  > Mask with GM </font>

In [11]:
quadrants=['LV','RV','LD','RD','interm','whole']
quadrants_files={'LV': config["main_dir"]+"/templates/PAM50/atlas_bin/GM_LV_bin.nii.gz", # GM left ventral horn
                'RV': config["main_dir"]+"/templates/PAM50/atlas_bin/GM_RV_bin.nii.gz",# GM righ ventral horn
                'LD': config["main_dir"]+"/templates/PAM50/atlas_bin/GM_LD_bin.nii.gz",  # GM left dorsal horn
                'RD': config["main_dir"]+"/templates/PAM50/atlas_bin/GM_RD_bin.nii.gz",#  GM righ dorsal horn
                'interm': config["main_dir"]+"/templates/PAM50/atlas_bin/GM_interm_bin.nii.gz", #GM left and right intermediate zone
                'whole': config["main_dir"]+"/templates/PAM50/atlas_bin/GM_whole_bin.nii.gz"}


for K in range(9,10):
    input_dir=config["main_dir"]+config["ica_root"]+ '/'+structure +'/K_'+ str(K) + '/comp_bin/'
    output_dir=config["main_dir"]+config["ica_root"]+ '/'+structure +'/K_'+ str(K) + '/comp_rois/'
    for k in range(len(sorted(glob.glob(input_dir + '*_k_*')))):
        for quadrant in quadrants:
            input_image=sorted(glob.glob(input_dir + '*_k_*'))[k]
            output_image=output_dir+os.path.basename(input_image).split('_bin.')[0]+'_'+quadrant+'_bin.nii.gz'
            run_proc('fslmaths {} -mas {} {}'.format(input_image,quadrants_files[quadrant],output_image)) # thre and binarize the image

fslmaths /cerebro/cerebro1/dataset/bmpd/derivatives/HealthyControls_project//ICA//spinalcord/K_9/comp_bin/zCanICA_22sbj_spinalcord_spinalcord_k_1_C7_bin.nii.gz -mas /cerebro/cerebro1/dataset/bmpd/derivatives/HealthyControls_project//templates/PAM50/atlas_bin/GM_LV_bin.nii.gz /cerebro/cerebro1/dataset/bmpd/derivatives/HealthyControls_project//ICA//spinalcord/K_9/comp_rois/zCanICA_22sbj_spinalcord_spinalcord_k_1_C7_LV_bin.nii.gz # in /cerebro/cerebro1/dataset/bmpd/derivatives/HealthyControls_project/ICA/code
fslmaths /cerebro/cerebro1/dataset/bmpd/derivatives/HealthyControls_project//ICA//spinalcord/K_9/comp_bin/zCanICA_22sbj_spinalcord_spinalcord_k_1_C7_bin.nii.gz -mas /cerebro/cerebro1/dataset/bmpd/derivatives/HealthyControls_project//templates/PAM50/atlas_bin/GM_RV_bin.nii.gz /cerebro/cerebro1/dataset/bmpd/derivatives/HealthyControls_project//ICA//spinalcord/K_9/comp_rois/zCanICA_22sbj_spinalcord_spinalcord_k_1_C7_RV_bin.nii.gz # in /cerebro/cerebro1/dataset/bmpd/derivatives/HealthyCo